In [22]:
import yaml
from box import ConfigBox


with open("config.yaml", "r") as ymlfile:
    config = yaml.safe_load(ymlfile)

config = ConfigBox(config)

In [23]:
!docker build -t us-central1-docker.pkg.dev/pebolas-sandbox/sample-model/titanic_model/deploy:latest --file /workspaces/sample-model/stacks/titanic_model/deploy/Dockerfile /workspaces/sample-model/stacks/titanic_model/deploy

[+] Building 0.0s (0/0)  docker:default
[+] Building 0.0s (0/0)  docker:default
[+] Building 0.0s (0/0)  docker:default
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 315B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.11-slim        0.2s
[+] Building 0.4s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 315B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.11-slim        0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s


In [24]:
from google.cloud import aiplatform
aiplatform.init(project=config.project_id, location=config.location)

from google.cloud.aiplatform.prediction import LocalModel

local_model = LocalModel(
    serving_container_image_uri=config.parameters.model_serving_container_image_uri,
    serving_container_health_route='/healthz',
    serving_container_predict_route='/predict'
)

In [25]:
with local_model.deploy_to_local_endpoint(
    artifact_uri="gs://sample-model-kubeflow-pipeline/titanic-pipeline/staging/",
    credential_path="/workspaces/sample-model/gcloud_credentials.json"
) as local_endpoint:
    health_check_response = local_endpoint.run_health_check()
    predict_response = local_endpoint.predict(
        request_file='/workspaces/sample-model/stacks/titanic_model/request.jsonl',
        headers={"Content-Type": "application/json"}
    )

In [26]:
health_check_response.content

b'{"status":"healthy"}\n'

In [27]:
predict_response.content

b'{"predictions":[{"result":[1,0,0,0]}]}\n'